In [1]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

In [2]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [4]:
from bs4 import BeautifulSoup
import requests

# Data
### We obtain the postal codes of the Greater Vancouver Region from public sources along with the GPS coordinates
### We will then compare this information using Foursquare API to see the types and numbers of businesses in each of the neighbourhood.

# Methodology
#### 1. put the postal codes into dataframe 
#### 2. pull the foursquare data using the coordinates of 100 business within 500 meters
#### 3. compare the neighbourhoods and group those.
#### 4. cluster those neighbourhoods by similar business establishment and map the clusters. 
#### 5. investigate the clusters


In [5]:
df = pd.read_csv("vancouver coordinates 2.csv") 

In [6]:
df.head()

,V3A,Langley Township,(Langley City)-Langley Township,49.100002,-122.657128
0,V4A,Surrey,Southwest-Surrey,49.044655,-122.869163
1,V5A,Burnaby,(Government Road / Lake City / SFU / Burnaby M...,49.276301,-122.946971
2,V6A,Vancouver,(Strathcona / Chinatown / Downtown Eastside)- ...,49.279935,-123.090704
3,V7A,Richmond,South- Richmond,49.134223,-123.099148
4,V3B,Port Coquitlam,Central- Port Coquitlam,49.259167,-122.746993


In [7]:
df.columns=["Postal Code", "City", "Neighbourhood", "Latitude","Longitude"]

In [8]:
df.head()

,Postal Code,City,Neighbourhood,Latitude,Longitude
0,V4A,Surrey,Southwest-Surrey,49.044655,-122.869163
1,V5A,Burnaby,(Government Road / Lake City / SFU / Burnaby M...,49.276301,-122.946971
2,V6A,Vancouver,(Strathcona / Chinatown / Downtown Eastside)- ...,49.279935,-123.090704
3,V7A,Richmond,South- Richmond,49.134223,-123.099148
4,V3B,Port Coquitlam,Central- Port Coquitlam,49.259167,-122.746993


In [9]:
df.shape

(89, 5)

In [10]:
df.tail()

,Postal Code,City,Neighbourhood,Latitude,Longitude
84,V7Y,Vancouver,(Pacific Centre)-Vancouver,49.282728,-123.118463
85,V2Z,Langley Township,Southwest-Langley Township,49.065755,-122.582949
86,V3Z,Surrey,Lower East-Surrey,49.048837,-122.693318
87,V5Z,Vancouver,(East Fairview / South Cambie)-Vancouver,49.233483,-123.120701
88,V6Z,Vancouver,(SW Downtown)-Vancouver,49.275944,-123.131166


In [11]:
vancouver_data=df

In [12]:
CLIENT_ID = 'PNBZRK0AATNMVG3M1CVPRL2IYCPPD5VI52ROWJKW53OZDUXM' # Foursquare ID
CLIENT_SECRET = 'IDM0XD24USYGN05IGYR5VI35ELC1UCWUA11JM3MF3AUDLVR3' #  Foursquare Secret
VERSION = '20180604'

In [13]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=1000
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
vancouver_venues = getNearbyVenues(names=vancouver_data['Neighbourhood'],
                                   latitudes=vancouver_data['Latitude'],
                                   longitudes=vancouver_data['Longitude']
                                  )

Southwest-Surrey
(Government Road / Lake City / SFU / Burnaby Mountain)-Burnaby
(Strathcona / Chinatown / Downtown Eastside)- Vancouver
South- Richmond
Central- Port Coquitlam
(Parkcrest-Aubrey / Ardingley-Sprott)-Burnaby
(NE Downtown / Gastown / Harbour Centre / International Village / Victory Square / Yaletown)-Vancouver
(Sea Island / YVR)-Richmond
South-Port Coquitlam
Northeast-Delta
(Burnaby Heights / Willingdon Heights / West Central Valley)- Burnaby
(Waterfront / Coal Harbour / Canada Place)-Vancouver
Northwest-Richmond
East-Delta
(Lakeview-Mayfield / Richmond Park / Kingsway-Beresford)-Burnaby
(SE West End / Davie Village)-Vancouver
Southwest-Richmond
East Central-Delta
(Cascade-Schou / Douglas-Gilpin)-Burnaby
(NW West End / Stanley Park)-Vancouver
Outer East-North Vancouver (district municipality)
Port Moody
(Maywood / Marlborough / Oakalla / Windsor)-Burnaby
(West Fairview / Granville Island / NE Shaughnessy)-Vancouver
Inner East-North Vancouver (district municipality)
(Suncre

In [15]:
vancouver_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Southwest-Surrey,49.044655,-122.869163,Potter's Ocean Park,49.040884,-122.867118,Flower Shop
1,Southwest-Surrey,49.044655,-122.869163,Crescent Park,49.049943,-122.862741,Park
2,Southwest-Surrey,49.044655,-122.869163,de VINE Skin,49.037769,-122.876598,Spa
3,Southwest-Surrey,49.044655,-122.869163,Esquires Coffee House,49.053296,-122.866983,Café
4,Southwest-Surrey,49.044655,-122.869163,Fieldstone Artisan Breads,49.053410,-122.867274,Bakery


In [16]:
vancouver_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
(Bentall Centre)-Vancouver,100,100,100,100,100,100
(Burnaby Heights / Willingdon Heights / West Central Valley)- Burnaby,58,58,58,58,58,58
(Cascade-Schou / Douglas-Gilpin)-Burnaby,20,20,20,20,20,20
(Central Kitsilano / Greektown)-Vancouver,87,87,87,87,87,87
(East Big Bend / Stride Avenue / Edmonds / Cariboo-Armstrong)-Burnaby,35,35,35,35,35,35
...,...,...,...,...,...,...
Southwest-Surrey,5,5,5,5,5,5
Upper East-Surrey,39,39,39,39,39,39
Upper West-Surrey,44,44,44,44,44,44


In [17]:
vancouver_venues_count=vancouver_venues.groupby('Neighbourhood').count()

In [18]:
vancouver_venues_count.sort_values(by=['Venue'], inplace=True, ascending=False)

In [19]:
vancouver_venues_count.head(20)

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
(Bentall Centre)-Vancouver,100,100,100,100,100,100
(NE Downtown / Gastown / Harbour Centre / International Village / Victory Square / Yaletown)-Vancouver,100,100,100,100,100,100
(SW Downtown)-Vancouver,100,100,100,100,100,100
(SE West End / Davie Village)-Vancouver,100,100,100,100,100,100
North-Richmond,100,100,100,100,100,100
(Pacific Centre)-Vancouver,100,100,100,100,100,100
(NW West End / Stanley Park)-Vancouver,100,100,100,100,100,100
(NW Shaughnessy / East Kitsilano / Quilchena)-Vancouver,100,100,100,100,100,100
(Waterfront / Coal Harbour / Canada Place)-Vancouver,100,100,100,100,100,100


In [20]:
# one hot encoding
vancouver_onehot = pd.get_dummies(vancouver_venues[['Venue Category']], prefix="", prefix_sep="")

In [21]:
vancouver_onehot['Neighbourhood'] = vancouver_venues['Neighbourhood'] 

In [22]:
fixed_columns = [vancouver_onehot.columns[-1]] + list(vancouver_onehot.columns[:-1])
vancouver_onehot = vancouver_onehot[fixed_columns]

vancouver_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,...,Vietnamese Restaurant,Warehouse Store,Water Park,Waterfront,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio,Zoo
0,Southwest-Surrey,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Southwest-Surrey,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Southwest-Surrey,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Southwest-Surrey,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Southwest-Surrey,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
vancouver_onehot.shape

(3697, 309)

In [24]:
vancouver_grouped = vancouver_onehot.groupby('Neighbourhood').mean().reset_index()
vancouver_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,...,Vietnamese Restaurant,Warehouse Store,Water Park,Waterfront,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio,Zoo
0,(Bentall Centre)-Vancouver,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020000,0.0,...,0.010000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.010000,0.0
1,(Burnaby Heights / Willingdon Heights / West C...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017241,0.0,...,0.017241,0.017241,0.0,0.0,0.0,0.017241,0.0,0.0,0.000000,0.0
2,(Cascade-Schou / Douglas-Gilpin)-Burnaby,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
3,(Central Kitsilano / Greektown)-Vancouver,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.011494,0.000000,0.0,0.0,0.0,0.011494,0.0,0.0,0.011494,0.0
4,(East Big Bend / Stride Avenue / Edmonds / Car...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.028571,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,Southwest-Surrey,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
85,Upper East-Surrey,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
86,Upper West-Surrey,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022727,0.0,...,0.045455,0.022727,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
87,West-Maple Ridge,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0


In [25]:
vancouver_grouped.shape

(89, 309)

In [26]:
num_top_venues = 5

for hood in vancouver_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = vancouver_grouped[vancouver_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----(Bentall Centre)-Vancouver----
                 venue  freq
0                Hotel  0.09
1         Dessert Shop  0.05
2           Food Truck  0.04
3          Coffee Shop  0.04
4  Japanese Restaurant  0.04


----(Burnaby Heights / Willingdon Heights / West Central Valley)- Burnaby----
                  venue  freq
0           Coffee Shop  0.10
1      Sushi Restaurant  0.05
2            Restaurant  0.05
3  Fast Food Restaurant  0.03
4              Pharmacy  0.03


----(Cascade-Schou / Douglas-Gilpin)-Burnaby----
                venue  freq
0            Bus Stop  0.15
1   Indian Restaurant  0.10
2      Sandwich Place  0.10
3         Coffee Shop  0.10
4  Chinese Restaurant  0.05


----(Central Kitsilano / Greektown)-Vancouver----
                           venue  freq
0                    Coffee Shop  0.11
1            Japanese Restaurant  0.05
2                           Park  0.05
3  Vegetarian / Vegan Restaurant  0.03
4                           Café  0.03


----(East Big Bend / Str

### put into pandas dataframe

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = vancouver_grouped['Neighbourhood']

for ind in np.arange(vancouver_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(vancouver_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,(Bentall Centre)-Vancouver,Hotel,Dessert Shop,Coffee Shop,Food Truck,Sandwich Place,Japanese Restaurant,Plaza,Gym,Café,Italian Restaurant
1,(Burnaby Heights / Willingdon Heights / West C...,Coffee Shop,Sushi Restaurant,Restaurant,Pharmacy,Burger Joint,Park,Fast Food Restaurant,Tailor Shop,Japanese Restaurant,Wine Shop
2,(Cascade-Schou / Douglas-Gilpin)-Burnaby,Bus Stop,Coffee Shop,Indian Restaurant,Sandwich Place,Bank,Chinese Restaurant,Candy Store,Sushi Restaurant,Gym,Latin American Restaurant
3,(Central Kitsilano / Greektown)-Vancouver,Coffee Shop,Park,Japanese Restaurant,Sushi Restaurant,Middle Eastern Restaurant,Café,Bakery,Tea Room,Vegetarian / Vegan Restaurant,Ice Cream Shop
4,(East Big Bend / Stride Avenue / Edmonds / Car...,Convenience Store,Sandwich Place,Bank,Grocery Store,Café,Chinese Restaurant,Thai Restaurant,Cantonese Restaurant,Music Store,Fast Food Restaurant


## cluster the neighbourhoods into 7 clusters

In [29]:
# set number of clusters
kclusters = 7

vancouver_grouped_clustering = vancouver_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(vancouver_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 0, 3, 0, 0, 0, 0, 0, 0], dtype=int32)

In [30]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)



In [31]:
neighborhoods_venues_sorted # check the data

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,(Bentall Centre)-Vancouver,Hotel,Dessert Shop,Coffee Shop,Food Truck,Sandwich Place,Japanese Restaurant,Plaza,Gym,Café,Italian Restaurant
1,3,(Burnaby Heights / Willingdon Heights / West C...,Coffee Shop,Sushi Restaurant,Restaurant,Pharmacy,Burger Joint,Park,Fast Food Restaurant,Tailor Shop,Japanese Restaurant,Wine Shop
2,0,(Cascade-Schou / Douglas-Gilpin)-Burnaby,Bus Stop,Coffee Shop,Indian Restaurant,Sandwich Place,Bank,Chinese Restaurant,Candy Store,Sushi Restaurant,Gym,Latin American Restaurant
3,3,(Central Kitsilano / Greektown)-Vancouver,Coffee Shop,Park,Japanese Restaurant,Sushi Restaurant,Middle Eastern Restaurant,Café,Bakery,Tea Room,Vegetarian / Vegan Restaurant,Ice Cream Shop
4,0,(East Big Bend / Stride Avenue / Edmonds / Car...,Convenience Store,Sandwich Place,Bank,Grocery Store,Café,Chinese Restaurant,Thai Restaurant,Cantonese Restaurant,Music Store,Fast Food Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...
84,0,Southwest-Surrey,Park,Spa,Flower Shop,Café,Bakery,Zoo,Filipino Restaurant,Farm,Farmers Market,Fast Food Restaurant
85,0,Upper East-Surrey,Coffee Shop,Restaurant,Fast Food Restaurant,Salon / Barbershop,Sushi Restaurant,Liquor Store,Salad Place,Gastropub,Gas Station,Bistro
86,3,Upper West-Surrey,Coffee Shop,Fast Food Restaurant,Pizza Place,Pub,Grocery Store,Bank,Vietnamese Restaurant,Gas Station,Gym / Fitness Center,Electronics Store
87,0,West-Maple Ridge,Pub,Gas Station,Bakery,Park,Track,Fast Food Restaurant,Restaurant,Grocery Store,Hotel,Financial or Legal Service


In [32]:
vancouver_data  #check data

,Postal Code,City,Neighbourhood,Latitude,Longitude
0,V4A,Surrey,Southwest-Surrey,49.044655,-122.869163
1,V5A,Burnaby,(Government Road / Lake City / SFU / Burnaby M...,49.276301,-122.946971
2,V6A,Vancouver,(Strathcona / Chinatown / Downtown Eastside)- ...,49.279935,-123.090704
3,V7A,Richmond,South- Richmond,49.134223,-123.099148
4,V3B,Port Coquitlam,Central- Port Coquitlam,49.259167,-122.746993
...,...,...,...,...,...
84,V7Y,Vancouver,(Pacific Centre)-Vancouver,49.282728,-123.118463
85,V2Z,Langley Township,Southwest-Langley Township,49.065755,-122.582949
86,V3Z,Surrey,Lower East-Surrey,49.048837,-122.693318
87,V5Z,Vancouver,(East Fairview / South Cambie)-Vancouver,49.233483,-123.120701


In [33]:

vancouver_merged = vancouver_data

# merge vancouver_grouped with vancouver_data to add latitude/longitude for each neighbuorhood
vancouver_merged = vancouver_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood', how='inner')

vancouver_merged.head() # check the last columns!

,Postal Code,City,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,V4A,Surrey,Southwest-Surrey,49.044655,-122.869163,0,Park,Spa,Flower Shop,Café,Bakery,Zoo,Filipino Restaurant,Farm,Farmers Market,Fast Food Restaurant
1,V5A,Burnaby,(Government Road / Lake City / SFU / Burnaby M...,49.276301,-122.946971,0,Bubble Tea Shop,Baseball Field,Burger Joint,Bus Line,Pizza Place,Bus Stop,Grocery Store,Park,Zoo,Financial or Legal Service
2,V6A,Vancouver,(Strathcona / Chinatown / Downtown Eastside)- ...,49.279935,-123.090704,0,Café,Park,Sandwich Place,Bar,Coffee Shop,Chinese Restaurant,Restaurant,Pizza Place,Brewery,Diner
3,V7A,Richmond,South- Richmond,49.134223,-123.099148,0,Pizza Place,Coffee Shop,Restaurant,Shopping Mall,Vietnamese Restaurant,Gym,Sporting Goods Shop,Portuguese Restaurant,Sushi Restaurant,Liquor Store
4,V3B,Port Coquitlam,Central- Port Coquitlam,49.259167,-122.746993,0,Fast Food Restaurant,Coffee Shop,Bank,Discount Store,Restaurant,Vietnamese Restaurant,Clothing Store,Eastern European Restaurant,Big Box Store,Sandwich Place


In [34]:
vancouver_merged.shape

(89, 16)

In [35]:
import folium 

In [36]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [37]:
from geopy.geocoders import Nominatim 

In [38]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [39]:
address = 'Vancouver, CA'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Vancouver are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Vancouver are 49.2608724, -123.1139529.


In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(vancouver_merged['Latitude'], vancouver_merged['Longitude'], vancouver_merged['Neighbourhood'], vancouver_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine the clusters

### Cluster 1

In [41]:
vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 0, vancouver_merged.columns[[2] + list(range(5, vancouver_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Southwest-Surrey,0,Park,Spa,Flower Shop,Café,Bakery,Zoo,Filipino Restaurant,Farm,Farmers Market,Fast Food Restaurant
1,(Government Road / Lake City / SFU / Burnaby M...,0,Bubble Tea Shop,Baseball Field,Burger Joint,Bus Line,Pizza Place,Bus Stop,Grocery Store,Park,Zoo,Financial or Legal Service
2,(Strathcona / Chinatown / Downtown Eastside)- ...,0,Café,Park,Sandwich Place,Bar,Coffee Shop,Chinese Restaurant,Restaurant,Pizza Place,Brewery,Diner
3,South- Richmond,0,Pizza Place,Coffee Shop,Restaurant,Shopping Mall,Vietnamese Restaurant,Gym,Sporting Goods Shop,Portuguese Restaurant,Sushi Restaurant,Liquor Store
4,Central- Port Coquitlam,0,Fast Food Restaurant,Coffee Shop,Bank,Discount Store,Restaurant,Vietnamese Restaurant,Clothing Store,Eastern European Restaurant,Big Box Store,Sandwich Place
...,...,...,...,...,...,...,...,...,...,...,...,...
82,(West Mount Pleasant / West Riley Park-Vancouver,0,Park,Coffee Shop,Bus Stop,Electronics Store,Accessories Store,Toy / Game Store,Sushi Restaurant,Fast Food Restaurant,Jewelry Store,Tea Room
83,Central-Richmond,0,Chinese Restaurant,Sushi Restaurant,Vietnamese Restaurant,Convenience Store,Gym,Bus Stop,Sandwich Place,Fast Food Restaurant,Park,Salon / Barbershop
84,(Pacific Centre)-Vancouver,0,Hotel,Coffee Shop,Dessert Shop,Sandwich Place,Japanese Restaurant,Seafood Restaurant,Restaurant,Italian Restaurant,Food Truck,Plaza
87,(East Fairview / South Cambie)-Vancouver,0,Sushi Restaurant,Fast Food Restaurant,Coffee Shop,Park,Sandwich Place,Bus Stop,Jewelry Store,Restaurant,Tea Room,Men's Store


### Cluster 2

In [42]:
vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 1, vancouver_merged.columns[[2] + list(range(5, vancouver_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
85,Southwest-Langley Township,1,Flower Shop,Construction & Landscaping,Zoo,Fish & Chips Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service


### Cluser 3

In [43]:
vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 2, vancouver_merged.columns[[2] + list(range(5, vancouver_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Northeast-Surrey,2,Trail,Flower Shop,Zoo,Financial or Legal Service,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant


### Cluster 4

In [44]:
vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 3, vancouver_merged.columns[[2] + list(range(5, vancouver_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,(Sea Island / YVR)-Richmond,3,Coffee Shop,Airport Service,Rental Car Location,Airport Lounge,Sandwich Place,Airport Terminal,Airport,Gift Shop,Bank,Duty-free Shop
8,South-Port Coquitlam,3,Coffee Shop,Grocery Store,Pizza Place,Bakery,Ice Cream Shop,Pharmacy,Pub,Trail,Sandwich Place,New American Restaurant
10,(Burnaby Heights / Willingdon Heights / West C...,3,Coffee Shop,Sushi Restaurant,Restaurant,Pharmacy,Burger Joint,Park,Fast Food Restaurant,Tailor Shop,Japanese Restaurant,Wine Shop
20,Outer East-North Vancouver (district municipal...,3,Pub,Food & Drink Shop,Bank,Grocery Store,Sporting Goods Shop,Park,Restaurant,Japanese Restaurant,Coffee Shop,Fast Food Restaurant
21,Port Moody,3,Coffee Shop,Sushi Restaurant,Pub,Grocery Store,Bank,Burger Joint,Bus Stop,Park,Sandwich Place,Farmers Market
26,(NW Shaughnessy / East Kitsilano / Quilchena)-...,3,Coffee Shop,Breakfast Spot,Furniture / Home Store,Restaurant,Yoga Studio,Japanese Restaurant,French Restaurant,Bakery,Vegetarian / Vegan Restaurant,Thai Restaurant
27,East Central-North Vancouver (district municip...,3,Coffee Shop,Pub,Arts & Crafts Store,Fast Food Restaurant,Sporting Goods Shop,French Restaurant,Brewery,Gas Station,Market,Farm
30,(Central Kitsilano / Greektown)-Vancouver,3,Coffee Shop,Park,Japanese Restaurant,Sushi Restaurant,Middle Eastern Restaurant,Café,Bakery,Tea Room,Vegetarian / Vegan Restaurant,Ice Cream Shop
31,North Central-North Vancouver (district munici...,3,Coffee Shop,Shopping Mall,Grocery Store,Bank,Fast Food Restaurant,Park,Liquor Store,Breakfast Spot,Juice Bar,Sushi Restaurant
33,Southeast-Delta,3,Coffee Shop,Bank,American Restaurant,Grocery Store,Athletics & Sports,Burger Joint,Gas Station,Sandwich Place,Sushi Restaurant,Restaurant


### Cluster 5

In [45]:
vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 4, vancouver_merged.columns[[2] + list(range(5, vancouver_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
86,Lower East-Surrey,4,Coffee Shop,Café,Fast Food Restaurant,Sandwich Place,Fair,Farm,Farmers Market,Field,Filipino Restaurant,Financial or Legal Service


### Cluster 6

In [46]:
vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 5, vancouver_merged.columns[[2] + list(range(5, vancouver_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
71,East-Langley Township,5,Exhibit,Intersection,Park,Zoo,Food Court,Filipino Restaurant,Falafel Restaurant,French Restaurant,Farm,Farmers Market


### Cluster 7

In [47]:
vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 6, vancouver_merged.columns[[2] + list(range(5, vancouver_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Northwest-Delta,6,Coffee Shop,Winery,Zoo,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service


In [48]:
van_1=vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 0, vancouver_merged.columns[[2] + list(range(5, vancouver_merged.shape[1]))]]

In [49]:
van_1.shape

(62, 12)

In [50]:
van_1.head()

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Southwest-Surrey,0,Park,Spa,Flower Shop,Café,Bakery,Zoo,Filipino Restaurant,Farm,Farmers Market,Fast Food Restaurant
1,(Government Road / Lake City / SFU / Burnaby M...,0,Bubble Tea Shop,Baseball Field,Burger Joint,Bus Line,Pizza Place,Bus Stop,Grocery Store,Park,Zoo,Financial or Legal Service
2,(Strathcona / Chinatown / Downtown Eastside)- ...,0,Café,Park,Sandwich Place,Bar,Coffee Shop,Chinese Restaurant,Restaurant,Pizza Place,Brewery,Diner
3,South- Richmond,0,Pizza Place,Coffee Shop,Restaurant,Shopping Mall,Vietnamese Restaurant,Gym,Sporting Goods Shop,Portuguese Restaurant,Sushi Restaurant,Liquor Store
4,Central- Port Coquitlam,0,Fast Food Restaurant,Coffee Shop,Bank,Discount Store,Restaurant,Vietnamese Restaurant,Clothing Store,Eastern European Restaurant,Big Box Store,Sandwich Place


In [51]:
van_4=vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 3, vancouver_merged.columns[[2] + list(range(5, vancouver_merged.shape[1]))]]

In [52]:
van_4.shape

(22, 12)

In [53]:
van_4.head()

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,(Sea Island / YVR)-Richmond,3,Coffee Shop,Airport Service,Rental Car Location,Airport Lounge,Sandwich Place,Airport Terminal,Airport,Gift Shop,Bank,Duty-free Shop
8,South-Port Coquitlam,3,Coffee Shop,Grocery Store,Pizza Place,Bakery,Ice Cream Shop,Pharmacy,Pub,Trail,Sandwich Place,New American Restaurant
10,(Burnaby Heights / Willingdon Heights / West C...,3,Coffee Shop,Sushi Restaurant,Restaurant,Pharmacy,Burger Joint,Park,Fast Food Restaurant,Tailor Shop,Japanese Restaurant,Wine Shop
20,Outer East-North Vancouver (district municipal...,3,Pub,Food & Drink Shop,Bank,Grocery Store,Sporting Goods Shop,Park,Restaurant,Japanese Restaurant,Coffee Shop,Fast Food Restaurant
21,Port Moody,3,Coffee Shop,Sushi Restaurant,Pub,Grocery Store,Bank,Burger Joint,Bus Stop,Park,Sandwich Place,Farmers Market
